<img src='logo.png' style='display: block;height: 61px;float: left;padding: .75rem 1.25rem;'>

# Desafío - Inferencia de tópicos con EM
* Para realizar este desafío debes haber revisado la lectura y videos correspondiente a la unidad.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminado el desafío, comprime la carpeta y sube el `.zip` a la seccióncorrespondiente.

## Descripción

* En esta sesión trabajaremos con una serie de base de datos sobre letras musicales de distintos artistas. Cada uno de los `csv` se encuentra en la carpeta `dump`.
* Cada `csv` tiene el nombre del artista a analizar. Los archivos contienen el nombre del artista, elgénero musical del artista, el nombre de la canción y las letras.
* En base a esta información, el objetivo del ejercicio es generar un modelo probabilístico quepueda identificar el género musical más probable dado la letra de una canción.
* Para ello implementaremos un modelo conocido como Latent Dirichlet Allocation que hace uso de una variante del algoritmo EM para inferir clases latentes a partir de una matriz dedocumentos.

## Ejercicio 1: Preparar el ambiente de trabajo

* Importe los módulos `numpy`, `pandas`, `matplotlib`, `seaborn`, `glob` y `os` siguiendo las buenas prácticas. Los últimos dos módulos permitirán realizar la importación de múltiplesarchivos dentro de la carpeta dump.
* Para ello genere un objeto que guarde en una lista todos los archivos alojados en dumputilizando `glob.glob` y `os.getcwd()` para extraer las rutas absolutas. Posteriormente genere un objeto `pd.DataFrame` que contenga todos los csv.
* Asegúrese de eliminar la columna `Unnamed: 0` que se genera por defecto.

> Se utilizan librerías bases para el tratamiento de datos y algunos aspectos básicos de cálculo y gráficos.

In [1]:
# Se importa libreria para el manejo de bases de datos
import pandas as pd
# Se importa libreria para el manejo de operaciones de cálculo
import numpy as np
# Se importa libreria para el manejo de gráficos
import matplotlib.pyplot as plt
# Se importa libreria para manejo de funciones estadisticas y probabilidades
import seaborn as sns
# Librería para visualizar patrones de datos perdidos
import missingno as msngo
# Se importa libreria para el menejo de warning
import warnings
# Se importa libreria para el menejo de expresiones regulares
import glob, os

> Definimos algunos aspectos de ambiente y valores por defecto.

In [3]:
# Por defecto, matplotlib crea una figura en una ventana separada.
# podemos hacer que las figuras aparezcan en línea dentro del notebook; lo hacemos ejecutando:
%matplotlib inline
# Se ignoran los warning para evitar suciedad en la ejecución
warnings.filterwarnings(action='ignore')
# Se define el estilo de gráficos a usar
plt.style.use('seaborn-pastel')
# Se define el tamaño de los paños de los gráficos por defecto
plt.rcParams['figure.figsize'] = (10, 6)
# Dado que vamos a supervisar datos no limitaremos la cantidad de columnas a mostrar en el despliegue del dataframe
pd.options.display.max_columns = None

> Se utiliza <strong>librería propia</strong> que continen funciones auxiliares, necesarias para el desarrollo de desafíos.

In [4]:
# Se importa libreria con funciones auxiliares
import ancilliary_funcs as afx

> Se utilizan librerías <strong>sklearn</strong> para el tratamiento de escala de valores de los atributos y seleccion de set de datos:

In [5]:
# Método para contar palabras y vectorizar
from sklearn.feature_extraction.text import CountVectorizer

> Carga de nuestra base de datos

In [6]:
# Generamos el listado de archivos con su path con la librería glob y lo
# almacenamos en la lista file_list
file_list = glob.glob(os.getcwd() + '/dump/*.csv')

In [7]:
# Iniciamos una lista vacia para cargar el contenido de cada archivo csv
append_csv = []
# Recorremos el listado de archivos, lo cargamos en memoria con read_csv y
# lo agregamos a la lista appappend_csv
for filename in file_list:
    append_csv.append(pd.read_csv(filename, index_col=None, header=0)
                      .drop(columns = 'Unnamed: 0')
                     )

In [8]:
# Se concatenan todos los objetos DataFrames almacenados en la lista append_csv en 1 solo DataFrame
df = pd.concat(append_csv)
# Se asigna un nombre a cada columna del DataFrame
df.columns = ['Artis', 'Genre', 'Song', 'Lyrics']

In [10]:
# Inspeccionamos los datos
df.sample(5)

,Artis,Genre,Song,Lyrics
204,Kiss,rock,When Lightning Strikes,"Alright \n It's my move, the ground's shakin' ..."
417,Eminem,hiphop,Rap Name - (Obie Trice),The rap game hip hop 101 the hardest 9 to 5 yo...
39,The Doors,rock,Celebration Of The Lizard (An Experiment/Work ...,Lions in the street and roaming \n Dogs in hea...
70,Iron Maiden,metal,The Prophecy,Now that I know that the right time has come \...
97,The Beatles,rock,Sgt. Pepper's Lonely Hearts Club Band,It was twenty years ago today \n That Sgt. Pep...


In [9]:
# Revisamos las dimensiones de la base de datos
df.shape

(9489, 4)

> La base de datos se compone de 9489 observaciones con 4 atributos. 

## Ejercicio 2: Matriz de ocurrencias

* Importe la clase `CountVectorizer` dentro de los módulos `feature_extraction.text` de lalibrería `sklearn`.
* Aplique la clase para extraer las 5000 palabras más repetidas en toda la base de datos.
* Con la clase inicializada, incorpore las letras con el método `fit_transform` y guarde los resultados en un nuevo objeto

> Convertimos los textos en vectores de recuento de palabras.

In [20]:
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer_fit = count_vectorizer.fit_transform(df['Lyrics'])

In [21]:
words = count_vectorizer.get_feature_names()
words_count = count_vectorizer_fit.toarray().sum(axis=0)
df_words = pd.DataFrame({'word': words, 'count': words_count})
df_words_5000 = df_words.sort_values(by='count', ascending=False).head(5000)

In [22]:
df_words_5000.shape

(5000, 2)

## Ejercicio 3: Entrenamiento del Modelo

* Importe `sklearn.decomposition.LatentDirichletAllocatin` y `sklearn.model_selection.GridSearchCV`.
* Genere una búsqueda de grilla con los siguientes hiperparámetros:
  - n_components: [5, 10, 15].
  - learning_decay: [0.7, 0.5].
* Entrene la búsqueda de grilla con las letras en un formato vectorizado con CountVectorizer.
* Reporte brevemente cuál es la mejor combinación de hiperparámetros.

> Digresión: Latent Dirichlet Allocation